# An introduction to Altair
## Licia He, Elsie Lee, and Eytan Adar

School of Information, University of Michigan

## Getting Started

In this tutorial we're going to walk you through the construction of visualizations using Altair--a tool to encode data using a Grammar of Graphics implementation (one called Vega-Lite). [Altair](https://altair-viz.github.io/) is a Python API to [Vega-Lite](https://vega.github.io/vega-lite/). When we code in Altair we are ultimately producing a data structure that tells Vega-Lite what to draw. Whatever is doable in Vega-Lite should be doable in Altair so you can always look at examples in one and figure out how to generate them in the other ([Vega-Lite Examples](https://vega.github.io/vega-lite/examples/) and [Altair Examples](https://altair-viz.github.io/gallery/index.html)). 

As a side note, Vega-Lite is a lighter variant of [Vega](https://vega.github.io/). Altair and Vega-Lite are equally capable. Vega, on the other hand, is much more full-featured but there is no API in Python at the moment. 

Internally, we are describing the visualization in a JSON object--a kind of data structure that has lists and dictionaries. This will allow the rendering system (the thing that draws on the screen) to intepret your Grammar of Graphics specficiation. You can always ask Altair to produce the Vega-Lite JSON object (and you can experiment directly with this JSON object in the [Vega/Vega-Lite Online editor](https://vega.github.io/editor/#/custom/vega).

In addition to Python, there are other language layers for Vega-Lite. The most popular of which is the [Javascript library](https://github.com/vega/vega-lite-api). It's very similar to the Python version so it's fairly straightforward to translate. The reason we're going to prefer Altair over the Javascript implementation is largely due to the data analysis ecosystem we have in Python. Tools like Pandas make data manipulation, cleaning, and analysis much easier.

We're going to try to cover some highlights of using Altair here, but very much encourage you to make use of online resources. Specifically:

* A whole [online course](https://github.com/uwdata/visualization-curriculum) by the creators of Vega-Lite. In fact, they show you have to use both Altair and the Javascript API.
* The [Altair tutorial](https://github.com/altair-viz/altair-tutorial) from the creator of Altair, Jake Vanderplas

Both will be useful resources if you get stuck or need more examples.

This particular tutorial will not cover interactivity or really sophisticated filtering/transformation. For the most part, we will either use data that's already in a form we want or get it into shape using Pandas.

We will be using two datasets here, the standard "cars" dataset (which has miles per gallon, cost, country of origin, horse power, etc.) and an ["indicators" dataset](http://www.bigcitieshealth.org/) that has various pieces of data for many cities in the US over time. Everything from demographics to the number of food poisonings. We've cleaned up the data a little bit for this exercise but you can grab the full data from [this github](https://github.com/altair-viz/vega_datasets/tree/master/vega_datasets/_data).

In this tutorial we will often slice-and-dice data to it's suitable for the examples we want to show. We will do this using [Pandas](https://pandas.pydata.org/pandas-docs/stable/index.html), a data manipulation library. Each block of code contains comments that explain the Pandas operations being used.

## 1. How we build a visualization

In [1]:
# 1.0 load up the libraries we need
import pandas as pd
import altair as alt

import pprint
pp = pprint.PrettyPrinter(indent=4)

Before we start building a chart, it will be helpful to understand what the Altair library does. As you write code in Altair, the system translates your code to the JSON format that Vega-Lite understands (and can render).

For example, let's say we want to use color to encode the transmission type of the car (it's either "automatic" or "manual", 1 or 0). To do so, we we use the "Color" method of the Altair library.

Run the code below to see what happens:

In [5]:
# 1.1 - The Altair code for producing a Color encoding on
# the transmission variable

alt.Color('transmission', type = 'nominal') # the JSON that Vega-Lite understands

Color({
  shorthand: 'transmission',
  type: 'nominal'
})

What you are seeing is the JSON corresponding to "snippet" of GoG Vega-Lite code. Remember, we need to describe: <font color='#55AAE7'>data</font>, <font color="#7F7C1C">encoding/channels</font>, and <font color="#AC491A">marks</font>. Here, we're describing the <font color='#55AAE7'>data</font> ("transmission" which is a "nominal" variable) and the <font color="#7F7C1C">encoding/channel</font> (color). In English: "encode the <font color='#55AAE7'>nominal transmission</font> variable using <font color="#7F7C1C">color</font>."

A couple of things to note.

First, although Vega-Lite and Altair will try to infer the type of the data (is transmission ordinal? nominal? time?) we can override the decision. In this case, our data file uses 0 and 1 to indicate manual or automatic. So we want to make sure that Altair understands that it's a nominal cateogry and not ordinal or quantiative.

This is important: Altair/Vega-Lite will use defaults or its best guess unless you override it. Most often you will be happy with those defaults and best guesses but there will be times when you want to force the system to do something else (use different colors, transform the data in a certain way, use different kinds of axes, etc.).

Second, notice we haven't actually loaded any data yet. Altair won't give you error messages until you try to render the Chart.  Here's an example that is encoding the MPG (miles per gallon) using the X coordinate.

Third, the quotes around the variables are in single quotes ' ', but they are interchangeable with double quotes " " which work exactly the same.

In [6]:
#1.2 Encoding MPG on the "X" channel

alt.X('MPG')

X({
  shorthand: 'MPG'
})

Ok, let's try to build an actual visualization. First, let's load some data:

In [8]:
#1.3 Let's grab some data and take a look

mtcars = pd.read_csv("https://raw.githubusercontent.com/eytanadar/si649public/master/lab3/data/mtcars.csv")

# Grab a few rows at random

mtcars.sample(5)

,model,MPG,cylinders,displacement,HP,rear_axle_ratio,weight,qsec,vs,transmission,gears,carb
7,Merc 240D,24.4,4,146.7,62,3.69,3.19,20.00,1,0,4,2
11,Merc 450SE,16.4,8,275.8,180,3.07,4.07,17.40,0,0,3,3
2,Datsun 710,22.8,4,108.0,93,3.85,2.32,18.61,1,1,4,1
12,Merc 450SL,17.3,8,275.8,180,3.07,3.73,17.60,0,0,3,3
23,Camaro Z28,13.3,8,350.0,245,3.73,3.84,15.41,0,0,3,4


Remember, to get a reasonable chart we need we need to describe what we want to use <font color='#55AAE7'>data</font>, <font color="#7F7C1C">encoding/channels</font>, and <font color="#AC491A">marks</font>.

To pick the kind of visualization we want, we need to make the right choice on the "mark." Do we want points (for a scatter plot)? lines (for a time series)? or bars (for a bar chart or heatmap)? See your choices [here](https://altair-viz.github.io/user_guide/marks.html).

Let's start easy. Let's make a scatterplot. We will tell Altair that we want a new Chart and that we will be using the mtcars dataframe. We will also, at minimum, neeed to pick the kind of mark. Scatterplots use points (a dot for every car in this case), so here's the code:

In [10]:
#1.4  Encode the dataset using points

alt.Chart(mtcars).mark_point()

alt.Chart(...)

Not very interesting... what we see is something like this:
    
![A dot](https://raw.githubusercontent.com/eytanadar/si649public/master/lab3/assets/simplestscatter.png)

(If you try to click on the little icon you may not even be able to see anything because the cell will be cut off)

The chart is exactly what we asked for. There is one dot for every car in the dataset. Unfortunately, because we didn't say how to encode the data (or even what actual pieces of the data we wanted), Altair went with some defaults: all points were placed at coordinate 0,0 and rendered with the default style (a blue outlined circle).

If you're curious you can take a look at what JSON Altair generated for this call:

In [15]:
#1.5  What does the Vega-lite JSON look like?
# alt.Chart(mtcars).mark_point().to_json() ---- ugly
pp.pprint(alt.Chart(mtcars).mark_point().to_json()) # ---- nicer version of json (pretty print)

('{\n'
 '  "$schema": "https://vega.github.io/schema/vega-lite/v4.8.1.json",\n'
 '  "config": {\n'
 '    "view": {\n'
 '      "continuousHeight": 300,\n'
 '      "continuousWidth": 400\n'
 '    }\n'
 '  },\n'
 '  "data": {\n'
 '    "name": "data-9409cf149fcd97d1b95049972fed8558"\n'
 '  },\n'
 '  "datasets": {\n'
 '    "data-9409cf149fcd97d1b95049972fed8558": [\n'
 '      {\n'
 '        "HP": 110,\n'
 '        "MPG": 21.0,\n'
 '        "carb": 4,\n'
 '        "cylinders": 6,\n'
 '        "displacement": 160.0,\n'
 '        "gears": 4,\n'
 '        "model": "Mazda RX4",\n'
 '        "qsec": 16.46,\n'
 '        "rear_axle_ratio": 3.9,\n'
 '        "transmission": 1,\n'
 '        "vs": 0,\n'
 '        "weight": 2.62\n'
 '      },\n'
 '      {\n'
 '        "HP": 110,\n'
 '        "MPG": 21.0,\n'
 '        "carb": 4,\n'
 '        "cylinders": 6,\n'
 '        "displacement": 160.0,\n'
 '        "gears": 4,\n'
 '        "model": "Mazda RX4 Wag",\n'
 '        "qsec": 17.02,\n'
 '        "rear_a

Ok, let's go with something a bit more interesting. We'll build a scatterplot comparing miles per gallon to car weight but let's just start with one variable, MPG, that we'll put on the x-axis.  Abstractly:

* <font color="#AC491A">Mark:    Point</font>
* <font color='#55AAE7'>Data:</font>
    * <font color='#55AAE7'>MPG:		quantitative</font>
* <font color="#7F7C1C">Encoding:</font>
    * <font color="#7F7C1C">MPG:		x position</font>


Or in English: "Draw a 1-D scatterplot with a point for every car where the miles per gallon of the car determine the placement of the point."

Or in code...

In [16]:
#1.6  Show MPG on the x-axis

alt.Chart(mtcars).mark_point().encode(
    alt.X('MPG', type = 'quantitative')
)

alt.Chart(...)

To break this apart: We grabbed the Altair library (`alt`) and told it to create a chart using the mtcars data frame (`.Chart(mtcars)`). We then said, for this chart use points as the mark style (`.mark_point`). We want to encode the points using the X location (`.encode(alt.X(`) as determined by the miles per gallon (`'MPG'`) variable which is a quantitative variable (`,type='quantitative'))`).

A side note: strictly speaking, we could have used `mark_circle` instead of `mark_point` if all we ever wanted were circle shapes. We're going to switch the shape later so we'll just keep using `mark_point`.

Notice that we get a nice visualization with an axis and axis label and we didn't need to do much work to get it. In fact, we could even do less work, there's a shortcut to specify the variable type (the `:Q` at the end of the variable name):

In [17]:
#1.7  We can also use a shortcut to indicate the variable type

alt.Chart(mtcars).mark_point().encode(
    alt.X('MPG:Q')
)

alt.Chart(...)

You might run into a few alternative versions of encoding. The ```alt.X(...)``` variant will come in handy later when we need to set additional parameters (e.g. bins, axis properties). But the others are great shorthand if you're happy with defaults. 

```Python
alt.Chart(mtcars).mark_point().encode(
    alt.X('MPG')
)```

```Python
alt.Chart(mtcars).mark_point().encode(
    x='MPG'
)```

```Python
alt.Chart(mtcars).mark_point().encode(
    x='MPG:Q'
)```

All these versions will produce the same visualization. The longest form has extra arguments that we will later use to have finer control of the look and feel or the transformations on the data. But you may see all these variants out in the wild.

Last step: let's make this a true scatter plot with 2 dimensions. We will use the weight for the Y-axis.

In [18]:
# 1.8  Visualize MPG and weight on a scatter plot

alt.Chart(mtcars).mark_point().encode(
    x = 'MPG', 
    y = 'weight'
)

alt.Chart(...)

## 2. Variants

Let's try some basic variants on the plot. 

Maybe we want to compare different transmission types. For this, we can use the color encoding for the transmission data: 

In [28]:
#2.1 MPG, weight, vs. transmission

alt.Chart(mtcars).mark_point().encode(
    x = 'MPG',
    y = 'weight',
    color = 'transmission'
)

alt.Chart(...)

This, unfortunately, isn't quite right. Transmission in our dataset is 0 or 1. Altair understood this to mean that the data was quantiative and encoded color on the default blue gradient. We want transmission to be a categorical or nominal variable so we can use:

In [29]:
#2.2 Fix a wrong inference, transmission is nominal

alt.Chart(mtcars).mark_point().encode(
    x = 'MPG',
    y = 'weight',
    color = 'transmission:N' # ---- specify that transmission is nominal
)

alt.Chart(...)

This would be a bit easier to read if the dots were filled so we could better see the colors. Luckily we can easily override the "global" properties of points to make them filled (`.mark_point(filled=True)`). Alternatively, you could use `.mark_circle`, which is a filled `.mark_point`. You can read more about this and see more options on the [Altair website](https://altair-viz.github.io/user_guide/customization.html).

In [30]:
#2.3 Change the look of the marks (filled)

alt.Chart(mtcars).mark_point(filled = True).encode(
    x = 'MPG',
    y = 'weight',
    color = 'transmission:N' # ---- specify that transmission is nominal
)

alt.Chart(...)

Let's say we're working with a black and white printer so color isn't the best encoding. Modify the chart definition to use shape instead of color (you might want to sneak a peak at the different encoding channels you have access to on [Altair site](https://altair-viz.github.io/user_guide/encoding.html). Try to make the default size a bit larger so the points are easier to see.

## 3. Bars and Binning

Let's try to build some bar charts. These will require switching to a different kind of mark (`mark_bar` in this case).  Let's plot the MPG for each car:

In [31]:
#3.1  use the "bar" mark to plot model to MPG

alt.Chart(mtcars).mark_bar().encode(
    x = 'model',
    y = 'MPG'
)

alt.Chart(...)

It's a bit hard to read the text. See if you can rotate the plot:

In [32]:
#3.2 Rotate the chart -- just switch the axis!

alt.Chart(mtcars).mark_bar().encode(
    y = 'model',
    x = 'MPG'
)

alt.Chart(...)

Often bar charts are used to "bin" data. For example, let's say we want to plot a bar chart of the number of cars in our dataset that are either manual or automatic. To do this, we would need to bin based on transmission type. We could use Pandas to do this. Here's one way to do this:

In [38]:
#3.3  Bin the cars to count how many cars have a specific transmission
#     We'll do this in Pandas first

binned = mtcars.groupby('transmission').count().reset_index()[['transmission','model']]
binned = binned.rename(columns = {'model':'count'})
binned

,transmission,count
0,0,19
1,1,13


From this, we can build our bar chart:

In [41]:
#3.4 Plot transmission vs. Count

alt.Chart(binned).mark_bar().encode(
    x = 'transmission:N',
    y = 'count'
)


alt.Chart(...)

Because this kind of operation is quite common, the Grammar of Graphics (generally), and Altair (specifically), provide an easier way to do this. 

We can tell Altair that we would like to bin our data (the default behavior for a nominal variable is to bin based on the categorical value... in this case 0 or 1).

`x=alt.X('transmission:N',bin=True)`

This will group all cars into two buckets: 1 and 0. At that point, we have to tell Altair what we want to know about each of those buckets. In this case we just want to know how many cars of each type we have. So we can use the special operation `count()`:

In [42]:
#3.5  Now in Altair

alt.Chart(mtcars).mark_bar().encode(
    x = alt.X('transmission:N', bin = True),
    y = alt.Y('count()')
)

alt.Chart(...)

Note that we can ask lots of other things about data in each bin. For example, we can ask for the average MPG (replace `count()` with `mean(MPG)`).

Or we could ask for the max horse power (`max(HP)`) or the median cylinders or any number of other aggregations. (see more on the [Altair site](https://altair-viz.github.io/user_guide/transform.html)). Try some of your own here:

In [44]:
#3.6 Can compute different things for each bin
#    For example, the max HP for each transmission type

alt.Chart(mtcars).mark_bar().encode(
    x = alt.X('transmission:N', bin = True),
    y = alt.Y('max(HP)')
)

alt.Chart(...)

We can bin data based on numerical properties as well. For example, we can generate a histogram of weights for our cars:

In [45]:
#3.7  Generate a simple histogram on a Quantiative variable
#     bin on Weight and count how many in each bin

alt.Chart(mtcars).mark_bar().encode(
    x = alt.X('weight', bin = True),
    y = alt.Y('count()')
)

alt.Chart(...)

As with most things in Altair, bins can be modified by using a special function to generate additional grammar of graphics commands. Bins, in particular, can be modified with the [BinParams](https://altair-viz.github.io/user_guide/generated/core/altair.BinParams.html#altair.BinParams) method. A simple example is making more bins:

In [46]:
#3.8 We can control properties of the bin using the BinParams object

alt.Chart(mtcars).mark_bar().encode(
    x = alt.X('weight', bin = alt.BinParams(maxbins = 20)),
    y = alt.Y('count()')
)

alt.Chart(...)

## 4. Sorting

In addition to binning, Altair also supports other transformations. Sorting is one of the most useful ones and is fairly easy. Below, we take our original bar chart showing the MPG for each car and add a sort on the Y-axis (the cars). We use the `EncodingSortField` function indicating that we want the models sorted by MPG (in descending order). More information on sorting can be found [here](https://altair-viz.github.io/user_guide/generated/core/altair.EncodingSortField.html).

In [61]:
#4.1  Let's try to sort

alt.Chart(mtcars).mark_bar().encode(
    x = alt.X('MPG'),
    y = alt.Y('model',
        sort = alt.EncodingSortField(
            field = 'MPG',
            order = 'descending'
        )
    )
)          

alt.Chart(...)

## 5. Other marks

Before we leave our cars dataset, we illustrate one other mark type, `mark_rect` which can be used for heatmap style visualizations. This structure is useful when we want to bin in two different dimensions. For example, we might want to know how many cars have certain combinations of MPG and HP. This would be:

In [63]:
#5.1  Make a heatmap to see the combination of cars
#     with certain MPG/HP

alt.Chart(mtcars).mark_rect().encode(
    x = alt.X('MPG', bin = True),
    y = alt.Y('HP', bin = True),
    color = 'count()'
)

alt.Chart(...)

## 6. Compound Chart 

Another cool aspect of Altair is that it's easy to put visualizations together. This is called [compound chart](https://altair-viz.github.io/user_guide/compound_charts.html). For example, the "|" operation will put two charts side by side whereas the "&" will put one on top of the other. You can combine these in various ways.

In [65]:
#6.1 Put two bar charts side by side: 1) MPG v model and 2) HP v. model

model_mpg = alt.Chart(mtcars).mark_bar().encode(
    x = alt.X('MPG'),
    y = alt.Y('model')
)

model_hp = alt.Chart(mtcars).mark_bar().encode(
    x = alt.X('HP'),
    y = alt.Y('model')
)

model_mpg | model_hp

# alt.hconcat(model_mpg,model_hp) # alternative


alt.HConcatChart(...)

In [66]:
#6.2 One above the other
model_mpg = alt.Chart(mtcars).mark_bar().encode(
    x = alt.X('MPG'),
    y = alt.Y('model')
)

model_hp = alt.Chart(mtcars).mark_bar().encode(
    x = alt.X('HP'),
    y = alt.Y('model')
)

model_mpg & model_hp

# alt.vconcat(model_mpg,model_hp) # alternative

alt.VConcatChart(...)

In [67]:
#6.3 Force a common (shared) axis

(model_mpg | model_hp.resolve_scale(y = 'shared'))

alt.HConcatChart(...)

## 7. Styling

### 7.1 DIY Option

We end with a note that while many of the defaults of Altair are reasonable there are situations that we need to override these either for aesthetic or more practical reasons. Take a look at the Altair documentation. Chances are you can modify the chart to look the way you want.

In [75]:
#7.1 color, axis 

model_mpg = alt.Chart(mtcars).mark_bar(height = 5).encode(
    x = alt.X('MPG'),
    y = alt.Y('model', sort = alt.EncodingSortField(
        field = 'model',
        order = 'descending'))
)

model_hp = alt.Chart(mtcars).mark_bar(color = 'orange').encode(
    x = alt.X('HP'),
    y = alt.Y('model')
)

model_mpg | model_hp


alt.HConcatChart(...)

In [72]:
# Now plot it, side by side

(model_mpg | model_hp).resolve_scale(y = 'shared').properties(
    title = 'MPG and HP'
)

alt.HConcatChart(...)

### 7.1 Themes

Vega-lite has a number of built-in themes (you can also make your own if you want). These themes provide a shortcut to creating visualizations that are styled in specific ways. For example, you can make your visualization look like ggplot, 538, vox, etc. The list of current themes is on the [vega-themes](https://github.com/vega/vega-themes) site. The appeal of using themes is that they are "activated" for all future visualizations until you turn them off.

If you would like to make your own, you will need to define the theme and then "register" it (give it a name). A good blog post that documents this is [here](https://towardsdatascience.com/consistently-beautiful-visualizations-with-altair-themes-c7f9f889602).  You will basically be creating a JSON object that describes how you want things to look colors/fonts/etc. (actually, you'll be describing only the things that you want to change from the default).

In [74]:
# 7.2
alt.themes.enable('fivethirtyeight')
alt.Chart(mtcars).mark_point().encode(
    x = 'MPG',
    y = 'weight',
    color = 'transmission:N'
)

alt.Chart(...)

In [76]:
# 7.3 to go back to the default, use this

alt.themes.enable('default')
alt.Chart(mtcars).mark_point().encode(
    x = 'MPG',
    y = 'weight',
    color = 'transmission:N'
)

alt.Chart(...)

# Summary

* Altair is a powerful way to produce Vega-Lite
    * You should think in marks (what shape do I want and for what); 
    * data (what data do I care about?) 
    * encodings (what visual thing does the data get mapped to)
* Can often pre-process data in Pandas, but...
    * there are powerful built in functions in Altair
* If you're stuck, look at the documentation for Altair, Vega-Lite (and sometimes Vega)